In [1]:
import json
import sys

sys.path.append('..')
from my_keywords import Keyword_Base
from my_multithread import MultiThreading
from my_util import lemmatize_all, my_read, my_write

In [2]:
# Process keyword file

filtered_words = set(['can', 'it', 'work', 'in', 'parts', 'its', 'a','b','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])

stable_kw = []
unstable_kw = []
with open('../data/raw_data/keyword.csv', 'r') as f_in:
    for line in f_in:
        kw = line.split(',')[0]
        if '- ' in kw:
            continue
        splited = kw.replace('-', ' - ')
        reformed = lemmatize_all(splited)
        if reformed in filtered_words:
            continue
        if reformed == splited:
            stable_kw.append(kw)
        else:
            unstable_kw.append('%s\t%s' % (kw, reformed))

with open('../data/corpus/keyword_f.txt', 'w') as f_out:
    f_out.write('\n'.join(stable_kw))

with open('../data/log/unstable_keyword.txt', 'w') as f_out:
    f_out.write('\n'.join(unstable_kw))

In [3]:
# Generate word tree
keyword_base = Keyword_Base()
keyword_base.build_word_tree('../data/corpus/keyword_f.txt', '../data/corpus/wordtree.json')

Building word tree is accomplished with 72102 words added


In [6]:
# Extract sentence from small corpus (2 minute)
!sed -n '/^ *"abstract/p' ../data/raw_data/small_arxiv.json | sed 's/.*\"abstract\": \"  \(.*\)\\n\",.*/\1/;s/\\n/ /g;s/\$//g;s/\\/ /g;s/---*/, /g;s/([^)]*)//g;s/{[^)]*}//g;s/-/ - /g' | tr -s [:space:] | tr '[:upper:]' '[:lower:]' > ../data/corpus/small_sent.txt

python ../my_sent_tokenize.py ../data/corpus/small_sent.txt ../data/corpus/small_sent.txt

In [ ]:
# Extract sentence from big corpus
# def extract_sent(line:str):
#     if not line:
#         return None
#     jsonObj = json.loads(line)
#     para = jsonObj['abstract'].strip().replace('\n', ' ').replace('$', '').replace('--', ', ').replace('-', ' - ')
#     return para + '\n'
# multithreading = MultiThreading()
# content = multithreading.run(extract_sent, input_file='../data/raw_data/big_arxiv.json', output_file='../data/corpus/big_sent.txt', thread_num=20).split('\n')